# GIS Tile Sniffer

https://earthexplorer.usgs.gov/

The URL can be broken down as follows:
 - Base URL: https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/
 - Zoom level: 17
 - Y coordinate: 49345
 - X coordinate: 27278

In [ ]:
# Build Maps for a Region starting from lat long
from tqdm import tqdm
import requests
import numpy as np
import cv2
import math

def lat_lon_to_tile(lat, lon, zoom):
    # Convert lat/lon to Web Mercator
    x = (lon + 180) / 360
    y = (1 - np.log(np.tan(np.radians(lat)) + 1 / np.cos(np.radians(lat))) / np.pi) / 2

    # Convert Web Mercator to tile coordinates
    n = 1 << zoom  # 2^zoom, funky bitwise shift operator
    x_tile = int(x * n)
    y_tile = int(y * n)
    
    return (x_tile, y_tile, zoom)

def highlight_tile_perimeter(tile):
    tile[0, :] = [0, 0, 255]  # Top edge
    tile[-1, :] = [0, 0, 255]  # Bottom edge
    tile[:, 0] = [0, 0, 255]  # Left edge
    tile[:, -1] = [0, 0, 255]  # Right edge
    return tile


# Boundary coordinates
coordinates = """
38.317297, -76.556176
38.315948, -76.556573
38.315467, -76.553762
38.314709, -76.549363
38.314241, -76.546627
38.313698, -76.543423 
38.313310, -76.541096
38.315299, -76.540521
38.315876, -76.543613
38.318616, -76.545385
38.318626, -76.552061
38.317034, -76.552447
38.316742, -76.552945
"""

zoom = 19 # lowest zoom level

# Parse coordinates
coords = [line.strip().split(',') for line in coordinates.strip().split('\n')]
lats = [float(coord[0]) for coord in coords]
lons = [float(coord[1]) for coord in coords]

# Find bounding box
min_lat, max_lat = min(lats), max(lats)
min_lon, max_lon = min(lons), max(lons)

# Add a small buffer to ensure we capture all needed tiles
buffer = 0.0001  # adjust as needed
min_lat -= buffer
max_lat += buffer
min_lon -= buffer
max_lon += buffer

# Get corner tiles
top_left = lat_lon_to_tile(max_lat, min_lon, zoom)
bottom_right = lat_lon_to_tile(min_lat, max_lon, zoom)

# Calculate tile range
delta_X = bottom_right[0] - top_left[0] + 1
delta_Y = bottom_right[1] - top_left[1] + 1

print(f"Downloading area from ({min_lat}, {min_lon}) to ({max_lat}, {max_lon})")
print(f"Tile range: {delta_X}x{delta_Y} tiles")

first_tile = top_left

# classic LR scan across a row then down to next
for y in tqdm(range(delta_Y), desc="Downloading", position=0):
    for x in tqdm(range(delta_X), desc="Columns", position=1, leave=False):
        current_tile = [
            first_tile[0] + x,
            first_tile[1] + y,
            first_tile[2]
        ] # X,Y,Z index

        URL = f"https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{str(current_tile[2])}/{str(current_tile[1])}/{str(current_tile[0])}"
        
        response = requests.get(URL)
        assert response.status_code == 200

        nparr = np.frombuffer(response.content, np.uint8) # Convert the binary data to a numpy array
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR) # Decode the image using OpenCV
        
        # Uncomment the next line if you want to highlight the tile perimeter
        # img = highlight_tile_perimeter(img)

        # Save each image separately with z, y, x coordinates in the filename
        cv2.imwrite(f"map_z{current_tile[2]}_y{current_tile[1]}_x{current_tile[0]}.png", img)

print("All tiles have been downloaded and saved separately.")